<a href="https://colab.research.google.com/github/avatar220928/Colab-Github/blob/material/%E5%B0%8F%E5%B7%A5%E5%BB%A0_google%E7%9B%B8%E9%97%9C%E4%BF%82%E6%95%B8%E8%A1%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **引入需要的套件**

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn import datasets
import requests
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


# **從google mobility爬蟲mobility資料**

In [ ]:
import urllib.request as urllib2
#urllib2.urlopen 

import zipfile #zipfile.ZipFile


def DownloadTWCompany():
    # 檔案下載

    print('開始')
    downloadurl = urllib2.urlopen('https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip')
    zipcontent= downloadurl.read()
    with open("TWRAW.zip", 'wb') as a:
        a.write(zipcontent)
    print ("下載完成!")

    # 解壓縮檔案

    print ("資料解壓縮...")
    with zipfile.ZipFile(open('TWRAW.zip', 'rb')) as a:
        a.extractall(".") 

    
    print ("解壓縮完成!")

DownloadTWCompany()

mobility = pd.read_csv('2020_TW_Region_Mobility_Report.csv')
print('有讀進去!')

開始
下載完成!
資料解壓縮...
解壓縮完成!
有讀進去!


# **def: 刪除google sheet裡的nan值**

In [ ]:
def deletenan (sheets,a):
  A = sheets.col_values(a)
  #print(A)
  country_google_name = A.copy() #A是google_name的複製版
  for i in range(len(A)):
    if A[i]== '':
      country_google_name.remove(A[i])
  # print(country_google_name)
  return country_google_name

# **def: 將tinestamp格式轉換成string**

In [ ]:
def timestamp2string(a):
  #a = a.to_pydatetime()
  str1 = a.strftime('%Y-%m-%d')
  return str1

# **def: 切開日期list、指定日期**

In [ ]:
def split_list(date, n, n_2):
  a = 0
  if n[0]!= 1:
    for  i_6 in range(len(n_2)):
      a = a + n[i_6]
      yield date[a : a + n_2[i_6]]
  else: 
      yield date[0 : 1]

# **def: 整理google mobility資料和case資料並且算出correlation**

In [ ]:
import math

all_case= pd.read_csv('/content/drive/MyDrive/mobility移動資料/all_cases/owid-covid-data.csv') #讀入新增案例數

country_mobility_1 = []
country_mobility_2 = []

file_name = ['2020_{}_Region_Mobility_Report.csv', '2021_{}_Region_Mobility_Report.csv','2022_{}_Region_Mobility_Report.csv'] #設定要讀入的名稱

google_places=['retail_and_recreation_percent_change_from_baseline',
        'grocery_and_pharmacy_percent_change_from_baseline',
        'parks_percent_change_from_baseline',
        'transit_stations_percent_change_from_baseline',
        'workplaces_percent_change_from_baseline']

labels = [ 'retail', 'grocery', 'park', 'transit', 'workplaces']

#開始算corr和p-value

def googlecorrtable(country_name, country_test):
  for i in range(len(country_name)):
    print('現在進入到這個國家: '+country_name[i])
    corr=[]
    p=[]
    choose=[]
    new_country_mobility = pd.DataFrame()

    for i_5 in range(0,3):
        country_mobility = pd.read_csv(file_name[i_5].format(country_name[i]))
        new_country_mobility = new_country_mobility.append(country_mobility)
    # print('成功')

    choose = all_case[ 'location' ] == country_test[i]

    n = len(start_date[i])
    A = pd.DataFrame(np.zeros((n*5,4)),columns=['n','place','Coefficient_of_Correlation','p-value'])

    for ii in range(len(start_date[i])): 
      A['n'][ii*5:ii*5+5] = str(ii+1)  #A的第一欄填上第幾波

      new_country_mobility.set_index(pd.to_datetime(new_country_mobility['date'],format='%Y-%m-%d'),inplace=True) 
      mobility=new_country_mobility [new_country_mobility ["date"].between(start_date[i][ii],end_date[i][ii])] 

      mobility_1 = mobility[mobility.sub_region_1.isnull()] #取出mobility中所需要的部分
      mobility_2 = mobility_1[mobility_1.metro_area.isnull()]
      mobility_3 = mobility_2[mobility_2.iso_3166_2_code.isnull()]
      
      TW_case = all_case[choose]
      TW_case.set_index(pd.to_datetime(TW_case['date'],format='%Y-%m-%d'),inplace=True)
      TW_case = TW_case[TW_case['date'].between(start_date[i][ii],end_date[i][ii])]

      for iii in range(len(google_places)):
        
        aa = mobility_3[google_places[iii]]
        bb = TW_case['new_cases']
        # print(np.isnan(aa))
        # print(np.isnan(bb))
        # print(type(aa))

        check_for_nan_1 = aa.isnull().any().any()
        # print ('google_mobility有nan嗎?'+ str(check_for_nan_1))

        check_for_nan_2 = bb.isnull().any().any()
        # print ('case number有nan嗎?'+ str(check_for_nan_2))

        if str(check_for_nan_1)=='True' or str(check_for_nan_2)=='True': 
          nansite_1 = aa.index[np.where(np.isnan(aa))]  #找google的nan位置
          nansite_2 = bb.index[np.where(np.isnan(bb))]  #找case的nan位置
          # print(timestamp2string(nansite_1))
          # print(timestamp2string(nansite_2))
          for cc in range(len(nansite_2)):
            aa = aa.drop(pd.to_datetime(timestamp2string(nansite_2)[cc]))  #轉換資料型態
          for dd in range(len(nansite_1)):
            bb = bb.drop(pd.to_datetime(timestamp2string(nansite_1)[dd]))

          bb = bb[bb.notna()]
          aa = aa[aa.notna()]
          # print(len(aa))
          # print(len(bb))
          # print(aa)
          # print(bb)

        
        if (len(aa)!=0):
          ans = pearsonr(aa, bb) #算出皮爾森相關係數
          corr.append(ans[0])
          p.append(ans[1])

        else:
          corr.append(-99)
          p.append(-99)
        # print(corr)


        sum=[]
        for b in range(len(start_date[i])):
          sum=sum+labels
        A['place']=sum     #在A的place欄位填入google場所

      
      # print('算完corr和p-value',country_name[i])

    A['Coefficient_of_Correlation'] = corr #在A的Coefficient_of_Correlation欄位填入相關係數
    A['p-value'] = p            #在A的p-value欄位填入p-value
    print(A)
    A.to_csv(country_name[i]+'_Google_相關係數.csv',index=False)
    mobility_extracted = mobility[['date',\
                    'retail_and_recreation_percent_change_from_baseline',\
                    'grocery_and_pharmacy_percent_change_from_baseline',\
                    'parks_percent_change_from_baseline',\
                    'transit_stations_percent_change_from_baseline',\
                    'workplaces_percent_change_from_baseline',\
                    'residential_percent_change_from_baseline']]
    
    i+1 

    


In [ ]:
import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

urls='https://docs.google.com/spreadsheets/d/1Jxcr8PvzqyEOa3DDT7wh9sBgJDGoDTqqLvDoT1r9R10/edit#gid=337413139'

sheet_name = ['Europe', 'North America','South America','Asia','Africa','Oceania'] 
country_mobility_1 = gc.open_by_url(urls)

for i_1 in range(len(sheet_name)):
  sheet = country_mobility_1.worksheet(sheet_name[i_1])  #讀取國家全名的全部sheet

  #消除nan

  data=[]
  col=[ii for ii in range(1,7)]

  for b in col:
    c = deletenan(sheet,a = b)  #去除每條column的nan值
    data.append(c)

  for bb in range(len(data)):

    int_n = list(map(int, data[3]))  #讓list裡的str變成int
    int_n_2 = int_n.copy()
    int_n.insert(0, 0)

  start_date = list(split_list(data[4], int_n, int_n_2))
  end_date = list(split_list(data[5], int_n, int_n_2))

  country_name = data[1] #google mobility
  country_test = data[0] #case number

  googlecorrtable(country_name, country_test)

現在進入到這個國家: GB


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


    n       place  Coefficient_of_Correlation       p-value
0   1      retail                   -0.690155  1.048879e-18
1   1     grocery                   -0.704203  1.024628e-19
2   1        park                   -0.530260  2.832779e-10
3   1     transit                   -0.753871  8.070461e-24
4   1  workplaces                   -0.759467  2.416880e-24
5   2      retail                   -0.446839  9.933579e-09
6   2     grocery                    0.173148  3.409779e-02
7   2        park                   -0.772310  5.777552e-31
8   2     transit                   -0.544590  5.852659e-13
9   2  workplaces                    0.112946  1.687787e-01
10  3      retail                   -0.297998  4.944652e-04
11  3     grocery                   -0.376302  8.045437e-06
12  3        park                   -0.521741  1.196534e-10
13  3     transit                   -0.475358  7.391045e-09
14  3  workplaces                   -0.315154  2.200208e-04
15  4      retail                    0.4

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


    n       place  Coefficient_of_Correlation       p-value
0   1      retail                   -0.720203  5.088783e-14
1   1     grocery                   -0.249910  2.537211e-02
2   1        park                   -0.637444  2.044486e-10
3   1     transit                   -0.754955  6.019414e-16
4   1  workplaces                   -0.193738  8.508180e-02
5   2      retail                    0.001493  9.898594e-01
6   2     grocery                    0.104699  3.713476e-01
7   2        park                   -0.079726  4.965455e-01
8   2     transit                   -0.152225  1.923088e-01
9   2  workplaces                   -0.278803  1.542718e-02
10  3      retail                   -0.561241  3.822237e-11
11  3     grocery                   -0.302200  8.813251e-04
12  3        park                   -0.370153  3.697467e-05
13  3     transit                   -0.503524  6.180409e-09
14  3  workplaces                   -0.191937  3.732629e-02
15  4      retail                   -0.0

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


    n       place  Coefficient_of_Correlation       p-value
0   1      retail                   -0.255425  3.621851e-04
1   1     grocery                   -0.124632  8.582416e-02
2   1        park                   -0.268337  1.745805e-04
3   1     transit                   -0.218392  2.404156e-03
4   1  workplaces                    0.033311  6.473290e-01
5   2      retail                   -0.272612  2.704367e-03
6   2     grocery                   -0.274016  2.565182e-03
7   2        park                   -0.168682  6.667658e-02
8   2     transit                   -0.357501  6.567203e-05
9   2  workplaces                   -0.244685  7.320027e-03
10  3      retail                   -0.547867  7.356643e-15
11  3     grocery                   -0.444817  9.758109e-10
12  3        park                   -0.537598  2.870231e-14
13  3     transit                   -0.394638  8.465186e-08
14  3  workplaces                   -0.018181  8.128770e-01
15  4      retail                    0.2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


    n       place  Coefficient_of_Correlation       p-value
0   1      retail                    0.429770  2.438078e-07
1   1     grocery                    0.425256  3.354718e-07
2   1        park                    0.302413  4.033476e-04
3   1     transit                    0.438189  1.327711e-07
4   1  workplaces                    0.166722  5.510666e-02
5   2      retail                    0.198380  2.596078e-02
6   2     grocery                    0.231274  9.171289e-03
7   2        park                    0.065784  4.642578e-01
8   2     transit                    0.269645  2.262941e-03
9   2  workplaces                    0.294273  8.238100e-04
10  3      retail                   -0.172344  6.920596e-02
11  3     grocery                    0.155118  1.024482e-01
12  3        park                   -0.265706  4.632961e-03
13  3     transit                   -0.008501  9.291152e-01
14  3  workplaces                   -0.130746  1.694211e-01
15  4      retail                   -0.4

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
